In [1]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/california-housing-prices/housing.csv


In [2]:
data = pd.read_csv("/kaggle/input/california-housing-prices/housing.csv")
data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [3]:
data.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [4]:
data.fillna({"total_bedrooms":data["total_bedrooms"].mode()[0]}, inplace = True)

In [5]:
data.isnull().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

In [6]:
data["ocean_proximity"] = data["ocean_proximity"].map({"<1H OCEAN": 0, "INLAND":1, "NEAR OCEAN": 2, "NEAR BAY": 3, "ISLAND":4})

In [7]:
X = data.drop("median_house_value", axis = 1)
y = data["median_house_value"]

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X,y, random_state = 41, test_size = 0.2)

In [9]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

In [10]:
models = {
    "LinearRegression": (LinearRegression(), {}),
    "Ridge": (Ridge(), {"alpha": [0.1, 1, 10, 100]}),
    "Lasso": (Lasso(), {"alpha": [0.01, 0.1, 1, 10]}),
    "RandomForestRegressor": (RandomForestRegressor(), {"n_estimators": [10, 50, 100], "max_depth": [None, 10, 20]}),
    "GradientBoostingRegressor": (GradientBoostingRegressor(), {"n_estimators": [50, 100], "learning_rate": [0.01, 0.1, 0.2]}),
    "KNeighborsRegressor": (KNeighborsRegressor(), {"n_neighbors": [3, 5, 7], "weights": ["uniform", "distance"]}),
    "DecisionTreeRegressor": (DecisionTreeRegressor(), {"max_depth": [None, 5, 10, 20]})
}

In [11]:
best_models = {}
for name, (model, param_grid) in models.items():
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='r2', n_jobs=-1,verbose=3)
    grid_search.fit(X_train, y_train)
    
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    best_models[name] = (best_model, mse, r2)
    
    print(f"{name}: Best Params: {grid_search.best_params_}, MSE: {mse:.4f}, R²: {r2:.4f}")

Fitting 5 folds for each of 1 candidates, totalling 5 fits
LinearRegression: Best Params: {}, MSE: 4850094323.2341, R²: 0.6312
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Ridge: Best Params: {'alpha': 10}, MSE: 4849822551.3616, R²: 0.6313
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Lasso: Best Params: {'alpha': 10}, MSE: 4849959482.1183, R²: 0.6313
Fitting 5 folds for each of 9 candidates, totalling 45 fits
RandomForestRegressor: Best Params: {'max_depth': None, 'n_estimators': 100}, MSE: 2262066266.5413, R²: 0.8280
Fitting 5 folds for each of 6 candidates, totalling 30 fits
GradientBoostingRegressor: Best Params: {'learning_rate': 0.2, 'n_estimators': 100}, MSE: 2584414012.5840, R²: 0.8035
Fitting 5 folds for each of 6 candidates, totalling 30 fits
KNeighborsRegressor: Best Params: {'n_neighbors': 7, 'weights': 'distance'}, MSE: 9232886036.2322, R²: 0.2980
Fitting 5 folds for each of 4 candidates, totalling 20 fits
DecisionTreeRegressor: Best Params: {'

In [12]:
best_model_name = max(best_models, key=lambda k: best_models[k][2])
print(f"\nBest Model: {best_model_name} with R²: {best_models[best_model_name][2]:.4f}")


Best Model: RandomForestRegressor with R²: 0.8280
